In [3]:
from src.config.data_loader_config import DATA_LOADER_CONFIG, OPTIMIZER_CONFIG
from src.data_loader.loader import Dataloader
from src.model.model import Model, Models, LossFunctions
from src.trainer.predict import save_result
from utils.fix_seed import set_seed
import src.callback as callback
import pytorch_lightning as pl
import torch
import os

# Parameters 설정
batch_size = DATA_LOADER_CONFIG['batch_size']
shuffle = DATA_LOADER_CONFIG['shuffle']
learning_rate = OPTIMIZER_CONFIG['learning_rate']
max_epoch = OPTIMIZER_CONFIG['max_epoch']
num_workers: int = DATA_LOADER_CONFIG.get('num_workers', 4)  # num_workers 기본값 4로 설정
torch.cuda.empty_cache()
set_seed(0)

# 모델 선언
# select Model -> roberta_base, roberta_small, roberta_large, electra_base, electra_base_v3
model_name = 'roberat_base_trending'
model = Model(Models.roberta_base, learning_rate, LossFunctions.mse_loss)

# Dataloader 선언
dataloader = Dataloader(batch_size, shuffle, Models.roberta_base, check_aug=False, num_workers=num_workers)

# callback 정의
epoch_print_callback = callback.EpochPrintCallback()
checkpoint_callback = callback.ModelCheckpoint(model_name=model_name)
early_stopping = callback.EarlyStopping()
lr_monitor = callback.LearningRateMonitor()

trainer = pl.Trainer(
    accelerator='auto',
    devices='auto',
    max_epochs=max_epoch,
    callbacks=[lr_monitor, epoch_print_callback, checkpoint_callback, early_stopping],
    precision='16-mixed',
    deterministic=True
)

Seed set to 0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# 학습

In [4]:
trainer.fit(model=model, datamodule=dataloader)

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Nam

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The variance of predictions or target is close to zero. This can cause instability in Pearson correlationcoefficient, leading to wrong results. Consider re-scaling the input if possible or computing using alarger dtype (currently using torch.float16).
  warnings.warn(*args, **kwargs)  # noqa: B028
c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\projec

Epoch 0:  60%|█████▉    | 348/583 [00:21<00:14, 16.45it/s, v_num=40]

# 추론


In [ ]:
# 가장 좋은 모델 불러오기
best_model_path = checkpoint_callback.best_model_path
model = Model.load_from_checkpoint(best_model_path)
trainer.test(model=model, datamodule=dataloader)
# 추론
predictions = trainer.predict(model=model, datamodule=dataloader)

# 결과 저장
save_result(predictions, model_name, max_epoch)